<a href="https://colab.research.google.com/github/saadz-khan/StyleGAN-client/blob/master/Stylegan2_ada_client.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Training StyleGAN2 on Colab

Use Colab Pro+ ($49.99/month), Colab Pro ($10.00/month) to get a couple extra hours of training time in per session and do not run into low-RAM issues.

Recommended: Colab Pro ($10.00 per month)

# Setting up only once. Kindly check the option
<center>  

![image](https://media.giphy.com/media/IFRhbMJSy3IaNZulCk/source.gif?cid=790b76116b562dd18158c804953ae06343645e756f8ed52d&rid=source.gif&ct=g)


# Custom Training StyleGan2-ADA

StyleGAN2-ADA only work with Tensorflow 1. Run the next cell before anything else to make sure we’re using TF1 and not TF2.

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
!nvidia-smi

Sat Oct 23 09:31:36 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Install Repo to Google Drive

Colab is a little funky with training. I’ve found the best way to do this is to install the repo directly into your Google Drive folder.

First, mount your Drive to the Colab notebook: |

## Mounting Google Drive
So I’m actually gonna install my entire repo directly into your Google Drive. This will make the setup a little easier.

First, connect your Drive to Colab. Gif explains how to:

<p align="center">
  <img src="https://media.giphy.com/media/WKMU25D95cjqWw9GCS/source.gif?cid=790b7611b611ba7d6a09693279bea7a3e08cb1e976b1b827&rid=source.gif&ct=g" alt="animated-gif2" />
</p>  

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Installing and Updating the Repo
Next, run this cell. If you’re already installed the repo,  
It will skip the installation process and change into the repo’s directory. If you haven’t installed it, it will install all the files necessary.

In [4]:
import os
if os.path.isdir("/content/drive/My Drive/colab-sg2-ada"):
    %cd "/content/drive/My Drive/colab-sg2-ada/stylegan2-ada"
else:
    #install script
    %cd "/content/drive/My Drive/"
    !mkdir colab-sg2-ada
    %cd colab-sg2-ada
    !git clone https://github.com/dvschultz/stylegan2-ada
    %cd stylegan2-ada
    !mkdir downloads
    !mkdir datasets

/content/drive/My Drive/colab-sg2-ada/stylegan2-ada


In [5]:
%cd "/content/drive/My Drive/colab-sg2-ada/stylegan2-ada"
!git config --global user.name "test"
!git config --global user.email "test@test.com"
!git fetch origin
!git checkout origin/main -- train.py

/content/drive/My Drive/colab-sg2-ada/stylegan2-ada


## Adding the dataset (Do not run) For testing

In [6]:
!pip install kaggle-cli

     |████████████████████████████████| 74 kB 2.1 MB/s 
     |████████████████████████████████| 4.2 MB 19.5 MB/s 
     |████████████████████████████████| 49 kB 5.4 MB/s 
     |████████████████████████████████| 144 kB 49.8 MB/s 
     |████████████████████████████████| 111 kB 50.3 MB/s 
  Created wheel for kaggle-cli: filename=kaggle_cli-0.12.13-py3-none-any.whl size=9487 sha256=40604f5f800ac08e34c804d993315cbf9f650b073c3153ec60a8761b3ee7e25e
  Stored in directory: /root/.cache/pip/wheels/76/b7/49/f439aedefd1839a586a54a03f30d5b9a5320c703cbee1631be
  ERROR: Failed building wheel for lxml
  Running setup.py clean for lxml
  Created wheel for PrettyTable: filename=prettytable-0.7.2-py3-none-any.whl size=13714 sha256=e29dd4f4b6ca88e476f68fea5ee50f0ba40e3461932addddce1f649d775020fc
  Stored in directory: /root/.cache/pip/wheels/b2/7f/f6/f180315b584f00445045ff1699b550fa895d09471337ce21c6
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=889875a2bfa7a5ef

In [7]:
%cd /content/drive/My\ Drive/
!mkdir ./data
!mkdir ./data/robgon-abstract/
#!mkdir /content/drive/My\ Drive/data/abstract-art
#!cp kaggle.json /content/drive/My\ Drive/kaggle/

/content/drive/My Drive
mkdir: cannot create directory ‘./data’: File exists
mkdir: cannot create directory ‘./data/robgon-abstract/’: File exists


In [12]:
!cp /content/drive/MyDrive/Dataset/kaggle.json ~/.kaggle/
!cat ~/.kaggle/kaggle.json 
!chmod 600 ~/.kaggle/kaggle.json

{"username":"szk2001","key":"01f895c6fe20d41b16b7dcfaf51a64e8"}

In [ ]:
#!kaggle datasets download -d robgonsalves/abstract-paintings -p Dataset
!kaggle datasets download -d arnaud58/landscape-pictures-p Dataset

In [19]:
#!mkdir /content/drive/MyDrive/Dataset/robgon-abstract
#!unzip /content/drive/MyDrive/Dataset/abstract-paintings.zip -d /content/drive/MyDrive/Dataset/robgon-abstract
!rm -rf /content/drive/MyDrive/Dataset/abstract-paintings.zip

## Convert dataset to .tfrecords

**Note: You only need to do this once per dataset. If you have already run this and are returning to conntinue training, skip these cells.**

Next we need to convert our image dataset to a format that StyleGAN2-ADA can read from. There are two options here. You can upload your dataset directly to Colab (as a zipped file), or you can upload it to Drive directly and read it from there.

In [20]:
#if you manually uploaded your dataset to Colab, unzip it Only once
zip_path = "/content/abc.zip"

!unzip {zip_path} -d /content/

unzip:  cannot find or open /content/drive/MyDrive/Dataset/robgon-abstract, /content/drive/MyDrive/Dataset/robgon-abstract.zip or /content/drive/MyDrive/Dataset/robgon-abstract.ZIP.


Now that your image dataset is uploaded, we need to convert it to the `.tfrecords` format.

Depending on the resolution of your images and how many you have, this can take a while.

In [21]:
%cd /content/drive/MyDrive/colab-sg2-ada/stylegan2-ada
#update this to the path to your image folder
dataset_path = "/content/drive/MyDrive/Dataset/robgon-abstract"
#give your dataset a name
dataset_name = "robgon-abstract"

#you don't need to edit anything here
!python dataset_tool.py create_from_images ./datasets/{dataset_name} {dataset_path}

/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada
Loading images from "/content/drive/MyDrive/Dataset/robgon-abstract"
Creating dataset "./datasets/robgon-abstract"
dataset_tool.py:97: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tostring()]))}))
Added 850 images.


## Train a custom model

We’re ready to start training! There are numerous arguments to training, what’s listed below are the most popular options. To see all the options, run the following cell.

In [22]:
# Use them only for metrics and help with the arguments
# This is just some help arguments
!python train.py --help

usage: train.py [-h] --outdir DIR [--gpus INT] [--snap INT] [--seed INT] [-n]
                --data PATH [--res INT] [--mirror BOOL] [--mirrory BOOL]
                [--use-raw BOOL] [--metrics LIST] [--metricdata PATH]
                [--cfg {auto,11gb-gpu,11gb-gpu-complex,24gb-gpu,24gb-gpu-complex,48gb-gpu,48gb-2gpu,stylegan2,paper256,paper512,paper1024,cifar,cifarbaseline,aydao}]
                [--lrate FLOAT] [--ttur BOOL] [--gamma FLOAT] [--nkimg INT]
                [--kimg INT] [--topk FLOAT] [--aug {noaug,ada,fixed,adarv}]
                [--p FLOAT] [--target TARGET] [--initstrength INITSTRENGTH]
                [--augpipe {blit,geom,color,filter,noise,cutout,bg,bgc,bgcf,bgcfn,bgcfnc}]
                [--cmethod {nocmethod,bcr,zcr,pagan,wgangp,auxrot,spectralnorm,shallowmap,adropout}]
                [--dcap FLOAT] [--resume RESUME] [--freezed INT]

Train a GAN using the techniques described in the paper
"Training Generative Adversarial Networks with Limited Data".

optional

In [ ]:
#this name must EXACTLY match the dataset name you used when creating the .tfrecords file
dataset_name = "robgon-abstract"
#how often should the model generate samples and a .pkl file
snapshot_count = 4
#should the images be mirrored left to right?
mirrored = True
#should the images be mirrored top to bottom?
mirroredY = True
#metrics? 
metric_list = None
#augments
augs = "bg"

#
# this is the most important cell to update
#
# running it for the first time? set it to ffhq(+resolution)
# resuming? get the path to your latest .pkl file and use that
resume_from = "/content/drive/My\ Drive/colab-sg2-ada2/stylegan2-ada/results/00008-dante1024-mirror-mirrory-11gb-gpu-bg-resumecustom/network-snapshot-000160.pkl"

#don't edit this unless you know what you're doing :)
!python train.py --outdir ./results --snap={snapshot_count} --cfg=11gb-gpu --data=./datasets/{dataset_name} --augpipe={augs} --mirror={mirrored} --mirrory={mirroredY} --metrics={metric_list} --resume={resume_from} --augpipe="bg"

### While it’s training...
**Once the above cell is running you should be training!**

Don’t close this tab! Colab needs to be open and running in order to continue training. Every ~15min or so a new line should get added to your output, indicated its still training. Depending on you `snapshot_count` setting you should see the results folder in your Google drive folder fill with both samples (`fakesXXXXXx.jpg`) and model weights (`network-snapshot-XXXXXX.pkl`). The samples are worth looking at while it trains but don’t get too worried about each individual sample.

If you chose a metric, you will also see scores for each snapshot. Don’t obsess over these! they are a guide, it can go up or down slightly for each snapshot. What you want to see is a gradual lowering of the score over time.

Once Colab shuts off, you can Reconnect the notebook and re-run every cell from top to bottom. Make sure you update the `resume_from` path to continue training from the latest model.

## Training model from scratch

In [ ]:
#this name must EXACTLY match the dataset name you used when creating the .tfrecords file
dataset_name = "robgon-abstract"
#how often should the model generate samples and a .pkl file
snapshot_count = 4
#should the images be mirrored left to right?
mirrored = True
#should the images be mirrored top to bottom?
mirroredY = True
#metrics? 
metric_list = None
#augments
augs = "bg"

#
# this is the most important cell to update
#
# running it for the first time? set it to ffhq(+resolution)

#don't edit this unless you know what you're doing :)
!python train.py --outdir ./results --snap={snapshot_count} --cfg=auto --data=./datasets/{dataset_name} --augpipe={augs} --mirror={mirrored} --mirrory={mirroredY} --metrics={metric_list} --augpipe="bg"

tcmalloc: large alloc 4294967296 bytes == 0x559d90e62000 @  0x7f1461ba5001 0x7f145eda854f 0x7f145edf8b58 0x7f145edfcb17 0x7f145ee9b203 0x559d89d5d544 0x559d89d5d240 0x559d89dd1627 0x559d89dcbced 0x559d89d5f48c 0x559d89da0159 0x559d89d9d0a4 0x559d89d5dd49 0x559d89dd194f 0x559d89dcb9ee 0x559d89c9de2b 0x559d89dcdfe4 0x559d89dcb9ee 0x559d89c9de2b 0x559d89dcdfe4 0x559d89dcbced 0x559d89c9de2b 0x559d89dcdfe4 0x559d89d5eafa 0x559d89dcc915 0x559d89dcb9ee 0x559d89dcb6f3 0x559d89e954c2 0x559d89e9583d 0x559d89e956e6 0x559d89e6d163
tcmalloc: large alloc 4294967296 bytes == 0x559e90e62000 @  0x7f1461ba31e7 0x7f145eda846e 0x7f145edf8c7b 0x7f145edf935f 0x7f145ee9b103 0x559d89d5d544 0x559d89d5d240 0x559d89dd1627 0x559d89dcb9ee 0x559d89d5ebda 0x559d89dcd737 0x559d89dcb9ee 0x559d89d5ebda 0x559d89dcd737 0x559d89dcb9ee 0x559d89d5ebda 0x559d89dcd737 0x559d89d5eafa 0x559d89dcc915 0x559d89dcb9ee 0x559d89d5ebda 0x559d89dd0d00 0x559d89dcb9ee 0x559d89d5ebda 0x559d89dcd737 0x559d89dcbced 0x559d89d5f48c 0x559d89da